<a href="https://colab.research.google.com/github/ZahraKheyrandish/Supply-Chain-Optimization-Inventory-Analysis/blob/main/data_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install Faker

In [49]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

In [50]:
fake = Faker('en_US')
Faker.seed(1234)
np.random.seed(1234)
random.seed(1234)

In [51]:
def get_seasonal_factor(date: datetime) -> float:
    """
    Calculates a seasonal factor based on the month.
    Boosts sales in winter months, slightly dips in summer.

    Args:
        date (datetime): The date for which to calculate the seasonal factor.

    Returns:
        float: The seasonal factor.
    """
    month = date.month
    if month in [12, 1, 2]:
        return 1.5
    elif month in [7, 8]:
        return 0.8
    return 1.0


In [54]:
def generate_products_df(num_products: int, faker_instance: Faker) -> pd.DataFrame:
    """
    Generates a DataFrame of synthetic product data for a pharmaceutical company.

    Args:
        num_products (int): The number of unique products to generate.
        faker_instance (Faker): An instance of the Faker library for generating names.

    Returns:
        pd.DataFrame: A DataFrame containing product_id, product_name, product_category,
                      unit_price, avg_shelf_life_days, and storage_conditions.
    """
    product_ids = [f'PROD_{i:05d}' for i in range(1, num_products + 1)]
    product_categories = [
        'Antibiotics', 'Pain Relievers', 'Anti-Inflammatories', 'Vitamins & Supplements',
        'Cardiovascular', 'Gastrointestinal', 'Respiratory', 'Neurological',
        'Dermatological', 'Ophthalmic & Otic', 'Diabetic Care', 'Hormonal',
        'Herbal Remedies', 'Disposable Medical Devices'
    ]
    product_names = []
    for _ in range(num_products):
        if random.random() < 0.7:
            product_names.append(f'{random.choice(["Tablet", "Capsule", "Syrup", "Ointment", "Ampoule", "Drop", "Cream"])} {faker_instance.word()} {faker_instance.word()}')
        else:
            product_names.append(f'{"".join(random.choices("ABCDEFGHIJKLMNOPQRSTUVWXYZ", k=2))}-{random.randint(100,999)} {random.choice(["mg", "g", "ml", "unit"])} {faker_instance.word()}')

    unit_prices = np.round(np.random.lognormal(mean=np.log(20), sigma=0.8, size=num_products), 2)
    unit_prices = np.maximum(unit_prices, 1.00)
    shelf_life_days = np.random.choice([365, 730, 1095, 1460, 1825], size=num_products, p=[0.1, 0.4, 0.3, 0.1, 0.1])
    storage_conditions = np.random.choice(['Room Temperature', 'Cold Storage', 'Protect from Light/Moisture', 'Refrigerated (2-8°C)'], size=num_products, p=[0.6, 0.2, 0.1, 0.1])

    products_df = pd.DataFrame({
        'product_id': product_ids,
        'product_name': product_names,
        'product_category': np.random.choice(product_categories, size=num_products),
        'unit_price': unit_prices,
        'avg_shelf_life_days': shelf_life_days,
        'storage_conditions': storage_conditions
    })
    products_df['product_category'] = products_df['product_category'].astype('category')
    products_df['storage_conditions'] = products_df['storage_conditions'].astype('category')
    return products_df

In [55]:
def generate_warehouses_df(num_warehouses: int) -> pd.DataFrame:
    """
    Generates a DataFrame of synthetic warehouse data.

    Args:
        num_warehouses (int): The number of warehouses to generate.

    Returns:
        pd.DataFrame: A DataFrame containing warehouse_id, warehouse_name, warehouse_city,
                      latitude, longitude, and capacity_units.
    """
    warehouse_ids = [f'WH_{i:02d}' for i in range(1, num_warehouses + 1)]
    warehouse_locations = {
        'Tehran': {'lat': 35.6892, 'lon': 51.3890}, 'Mashhad': {'lat': 36.2605, 'lon': 59.6168},
        'Isfahan': {'lat': 32.6546, 'lon': 51.6670}, 'Tabriz': {'lat': 38.0805, 'lon': 46.2918},
        'Shiraz': {'lat': 29.6065, 'lon': 52.5414}, 'Ahvaz': {'lat': 31.3204, 'lon': 48.6720},
        'Karaj': {'lat': 35.8322, 'lon': 50.9667}, 'Rasht': {'lat': 37.2752, 'lon': 49.5891},
        'Kerman': {'lat': 30.2832, 'lon': 57.0671}, 'Bandar Abbas': {'lat': 27.1887, 'lon': 56.2829}
    }
    warehouse_names = [f'Central Warehouse {city}' for city in warehouse_locations.keys()]
    warehouse_latitudes = [loc['lat'] for loc in warehouse_locations.values()]
    warehouse_longitudes = [loc['lon'] for loc in warehouse_locations.values()]
    capacity_units = np.random.randint(1_000_000, 5_000_000, size=num_warehouses)
    capacity_units[0] = np.random.randint(4_000_000, 7_000_000)

    warehouses_df = pd.DataFrame({
        'warehouse_id': warehouse_ids,
        'warehouse_name': warehouse_names,
        'warehouse_city': list(warehouse_locations.keys()),
        'latitude': warehouse_latitudes,
        'longitude': warehouse_longitudes,
        'capacity_units': capacity_units
    })
    warehouses_df['warehouse_city'] = warehouses_df['warehouse_city'].astype('category')
    return warehouses_df

In [56]:
def generate_suppliers_df(num_suppliers: int, faker_instance: Faker) -> pd.DataFrame:
    """
    Generates a DataFrame of synthetic supplier data.

    Args:
        num_suppliers (int): The number of suppliers to generate.
        faker_instance (Faker): An instance of the Faker library for generating names.

    Returns:
        pd.DataFrame: A DataFrame containing supplier_id, supplier_name, supplier_location,
                      avg_lead_time_days, and reliability_score.
    """
    supplier_ids = [f'SUP_{i:03d}' for i in range(1, num_suppliers + 1)]
    supplier_names = [faker_instance.company() for _ in range(num_suppliers)]
    supplier_locations = [faker_instance.city() for _ in range(num_suppliers)]
    avg_lead_time_days = np.random.randint(5, 30, size=num_suppliers)
    reliability_scores = np.round(np.random.uniform(0.7, 0.99, size=num_suppliers), 2)

    suppliers_df = pd.DataFrame({
        'supplier_id': supplier_ids,
        'supplier_name': supplier_names,
        'supplier_location': supplier_locations,
        'avg_lead_time_days': avg_lead_time_days,
        'reliability_score': reliability_scores
    })
    suppliers_df['supplier_location'] = suppliers_df['supplier_location'].astype('category')
    return suppliers_df

In [57]:
def generate_purchase_orders_df(num_purchase_orders: int, products_df: pd.DataFrame, suppliers_df: pd.DataFrame, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    """
    Generates a DataFrame of synthetic purchase order data.

    Args:
        num_purchase_orders (int): The number of purchase orders to generate.
        products_df (pd.DataFrame): DataFrame of product master data.
        suppliers_df (pd.DataFrame): DataFrame of supplier master data.
        start_date (datetime): The start date for generating POs.
        end_date (datetime): The end date for generating POs.

    Returns:
        pd.DataFrame: A DataFrame containing po_id, po_date, supplier_id, product_id,
                      ordered_quantity, unit_cost, expected_delivery_date, actual_delivery_date,
                      and delivery_status.
    """
    po_data = []
    for i in range(num_purchase_orders):
        po_id = f'PO_{i:05d}'
        po_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

        product = products_df.sample(1).iloc[0]
        supplier = suppliers_df.sample(1, weights='reliability_score').iloc[0]

        base_quantity = np.random.randint(500, 5000)
        ordered_quantity = max(10, int(base_quantity / (product['unit_price'] / 10 + 1)))
        unit_cost = product['unit_price'] * np.random.uniform(0.5, 0.8)
        expected_delivery_date = po_date + timedelta(days=int(supplier['avg_lead_time_days']))

        delivery_delay_days = 0
        if random.random() < (1 - supplier['reliability_score']):
            delivery_delay_days = np.random.randint(1, 10)
            if random.random() < 0.2:
                delivery_delay_days = np.random.randint(10, 30)

        actual_delivery_date = expected_delivery_date + timedelta(days=int(delivery_delay_days))

        delivery_status = 'Delivered'
        if actual_delivery_date > end_date:
            delivery_status = 'Pending'
        elif actual_delivery_date > expected_delivery_date:
            delivery_status = 'Delayed'

        po_data.append({
            'po_id': po_id,
            'po_date': po_date,
            'supplier_id': supplier['supplier_id'],
            'product_id': product['product_id'],
            'ordered_quantity': ordered_quantity,
            'unit_cost': unit_cost,
            'expected_delivery_date': expected_delivery_date,
            'actual_delivery_date': actual_delivery_date,
            'delivery_status': delivery_status
        })

    purchase_orders_df = pd.DataFrame(po_data)
    purchase_orders_df['po_date'] = pd.to_datetime(purchase_orders_df['po_date'])
    purchase_orders_df['expected_delivery_date'] = pd.to_datetime(purchase_orders_df['expected_delivery_date'])
    purchase_orders_df['actual_delivery_date'] = pd.to_datetime(purchase_orders_df['actual_delivery_date'])
    purchase_orders_df['delivery_status'] = purchase_orders_df['delivery_status'].astype('category')
    return purchase_orders_df



In [58]:
def generate_customers_df(num_customers: int, faker_instance: Faker) -> pd.DataFrame:
    """
    Generates a DataFrame of synthetic customer (pharmacy) data.

    Args:
        num_customers (int): The number of customers to generate.
        faker_instance (Faker): An instance of the Faker library for generating names.

    Returns:
        pd.DataFrame: A DataFrame containing customer_id, customer_name, and customer_location.
    """
    customer_ids = [f'CUST_{i:04d}' for i in range(1, num_customers + 1)]
    customer_names = [faker_instance.company() + ' Pharmacy' for _ in range(num_customers)]
    customer_locations = [faker_instance.city() for _ in range(num_customers)]

    customers_df = pd.DataFrame({
        'customer_id': customer_ids,
        'customer_name': customer_names,
        'customer_location': customer_locations
    })
    customers_df['customer_location'] = customers_df['customer_location'].astype('category')
    return customers_df

In [59]:
def generate_sales_df(num_sales_records: int, products_df: pd.DataFrame, customers_df: pd.DataFrame, warehouses_df: pd.DataFrame, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    """
    Generates a DataFrame of synthetic sales data.

    Args:
        num_sales_records (int): The number of sales records to generate.
        products_df (pd.DataFrame): DataFrame of product master data.
        customers_df (pd.DataFrame): DataFrame of customer master data.
        warehouses_df (pd.DataFrame): DataFrame of warehouse master data.
        start_date (datetime): The start date for generating sales.
        end_date (datetime): The end date for generating sales.

    Returns:
        pd.DataFrame: A DataFrame containing sales_id, sales_date, customer_id, warehouse_id,
                      product_id, sold_quantity, sales_price, and delivery_time_hours.
    """
    sales_data = []
    for i in range(num_sales_records):
        sales_id = f'SALE_{i:06d}'
        sales_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))

        product = products_df.sample(1).iloc[0]
        customer = customers_df.sample(1).iloc[0]
        warehouse = warehouses_df.sample(1).iloc[0]

        base_sold_quantity = np.random.randint(5, 100)
        adjusted_sold_quantity = max(1, int(base_sold_quantity / (product['unit_price'] / 10 + 1)))

        seasonal_factor = get_seasonal_factor(sales_date)
        sold_quantity = max(1, int(adjusted_sold_quantity * seasonal_factor * np.random.uniform(0.8, 1.2)))

        sales_price = product['unit_price'] * sold_quantity
        delivery_time_hours = np.random.randint(4, 49)

        sales_data.append({
            'sales_id': sales_id,
            'sales_date': sales_date,
            'customer_id': customer['customer_id'],
            'warehouse_id': warehouse['warehouse_id'],
            'product_id': product['product_id'],
            'sold_quantity': sold_quantity,
            'sales_price': sales_price,
            'delivery_time_hours': delivery_time_hours
        })

    sales_df = pd.DataFrame(sales_data)
    sales_df['sales_date'] = pd.to_datetime(sales_df['sales_date'])
    return sales_df


In [60]:
def generate_returns_df(sales_df: pd.DataFrame, num_returns: int) -> pd.DataFrame:
    """
    Generates a DataFrame of synthetic product return data.

    Args:
        sales_df (pd.DataFrame): DataFrame of sales data.
        num_returns (int): The number of return records to generate.

    Returns:
        pd.DataFrame: A DataFrame containing return_id, sales_id, return_date, product_id,
                      warehouse_id, returned_quantity, and return_reason.
    """
    return_reasons = ['Damaged in transit', 'Incorrect item received', 'Customer changed mind', 'Expired product received']
    returns_data = []

    returned_sales = sales_df.sample(n=num_returns, replace=True).reset_index(drop=True) # Use replace=True if num_returns > len(sales_df)

    for index, row in returned_sales.iterrows():
        return_date = row['sales_date'] + timedelta(days=random.randint(1, 14))
        returned_quantity = random.randint(1, row['sold_quantity'])
        return_reason = random.choice(return_reasons)

        returns_data.append({
            'return_id': f'RET_{index:05d}',
            'sales_id': row['sales_id'],
            'return_date': return_date,
            'product_id': row['product_id'],
            'warehouse_id': row['warehouse_id'],
            'returned_quantity': returned_quantity,
            'return_reason': return_reason
        })

    returns_df = pd.DataFrame(returns_data)
    returns_df['return_date'] = pd.to_datetime(returns_df['return_date'])
    returns_df['return_reason'] = returns_df['return_reason'].astype('category')
    return returns_df

In [61]:
def simulate_inventory_and_waste(
    purchase_orders_df: pd.DataFrame,
    sales_df: pd.DataFrame,
    returns_df: pd.DataFrame,
    products_df: pd.DataFrame,
    warehouses_df: pd.DataFrame,
    start_date: datetime,
    end_date: datetime
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Simulates inventory levels over time, tracking receipts, issues, returns, and spoilage.
    Also generates records for stockouts and wasted inventory.

    Args:
        purchase_orders_df (pd.DataFrame): DataFrame of purchase orders.
        sales_df (pd.DataFrame): DataFrame of sales data.
        returns_df (pd.DataFrame): DataFrame of returns data.
        products_df (pd.DataFrame): DataFrame of product master data.
        warehouses_df (pd.DataFrame): DataFrame of warehouse master data.
        start_date (datetime): The overall start date for the simulation.
        end_date (datetime): The overall end date for the simulation.

    Returns:
        tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
            - inventory_df: DataFrame of inventory snapshots over time.
            - waste_df: DataFrame of wasted (expired) inventory.
            - stockout_df: DataFrame of stockout events.
    """
    inventory_events = []

    # Add Receipts from Purchase Orders
    for _, row in purchase_orders_df.iterrows():
        if row['delivery_status'] == 'Delivered':
            expiry_date = row['actual_delivery_date'] + timedelta(days=int(products_df[products_df['product_id'] == row['product_id']]['avg_shelf_life_days'].iloc[0]))
            inventory_events.append({
                'event_date': row['actual_delivery_date'],
                'product_id': row['product_id'],
                'warehouse_id': random.choice(warehouses_df['warehouse_id'].tolist()),
                'quantity_change': row['ordered_quantity'],
                'event_type': 'Receipt',
                'batch_number': f'BATCH_{row["po_id"]}',
                'expiry_date': expiry_date
            })

    # Add Issues from Sales
    for _, row in sales_df.iterrows():
        inventory_events.append({
            'event_date': row['sales_date'],
            'product_id': row['product_id'],
            'warehouse_id': row['warehouse_id'],
            'quantity_change': -row['sold_quantity'],
            'event_type': 'Issue',
            'batch_number': None, # Batch determined during simulation
            'expiry_date': None # Expiry determined during simulation
        })

    # Add Receipts from Returns
    for _, row in returns_df.iterrows():
        returned_product_shelf_life = int(products_df[products_df['product_id'] == row['product_id']]['avg_shelf_life_days'].iloc[0])
        expiry_date_return = row['return_date'] + timedelta(days=min(returned_product_shelf_life, 180))

        inventory_events.append({
            'event_date': row['return_date'],
            'product_id': row['product_id'],
            'warehouse_id': row['warehouse_id'],
            'quantity_change': row['returned_quantity'],
            'event_type': 'Return_Receipt',
            'batch_number': f'RET_BATCH_{row["return_id"]}',
            'expiry_date': expiry_date_return
        })

    inventory_events_df = pd.DataFrame(inventory_events)
    inventory_events_df['event_date'] = pd.to_datetime(inventory_events_df['event_date'])
    inventory_events_df = inventory_events_df.sort_values(by='event_date').reset_index(drop=True)
    inventory_events_df['event_type'] = inventory_events_df['event_type'].astype('category')


    inventory_records = []
    current_inventory = {} # Key: (warehouse_id, product_id, batch_number), Value: {'quantity': X, 'expiry_date': Y}
    waste_records = []
    stockout_records = []

    all_dates = sorted(inventory_events_df['event_date'].unique())

    for current_date_np in all_dates:
        current_date = pd.to_datetime(current_date_np)

        # --- Process Spoilage/Waste for the current date ---
        expired_batches = []
        for (wh_id, prod_id, batch_num), details in list(current_inventory.items()): # Use list() to iterate over a copy
            if details['quantity'] > 0 and details['expiry_date'] and details['expiry_date'] <= current_date:
                expired_batches.append(((wh_id, prod_id, batch_num), details['quantity']))

        for (wh_id, prod_id, batch_num), expired_qty in expired_batches:
            if current_inventory.get((wh_id, prod_id, batch_num), {}).get('quantity', 0) > 0:
                wasted_amount = current_inventory[(wh_id, prod_id, batch_num)]['quantity']
                waste_records.append({
                    'waste_date': current_date,
                    'warehouse_id': wh_id,
                    'product_id': prod_id,
                    'batch_number': batch_num,
                    'wasted_quantity': wasted_amount,
                    'reason': 'Expired'
                })
                current_inventory[(wh_id, prod_id, batch_num)]['quantity'] = 0

        # --- Process daily events (Receipts, Issues, Returns) ---
        daily_events = inventory_events_df[inventory_events_df['event_date'] == current_date].copy()

        for _, event in daily_events.iterrows():
            product_id = event['product_id']
            warehouse_id = event['warehouse_id']
            quantity_change = event['quantity_change']
            event_type = event['event_type']

            if event_type in ['Receipt', 'Return_Receipt']:
                batch_number = event['batch_number']
                expiry_date = event['expiry_date']
                key = (warehouse_id, product_id, batch_number)
                if key not in current_inventory:
                    current_inventory[key] = {'quantity': 0, 'expiry_date': expiry_date}
                current_inventory[key]['quantity'] += quantity_change
            elif event_type == 'Issue':
                remaining_to_sell = abs(quantity_change)

                available_batches = []
                for (wh_id, prod_id, batch_num), details in current_inventory.items():
                    if wh_id == warehouse_id and prod_id == product_id and details['quantity'] > 0:
                        available_batches.append((details['expiry_date'], batch_num, details['quantity']))

                available_batches.sort() # Sort by expiry date (FEFO)

                total_available_for_product = sum(current_inventory.get((wh_id, prod_id, b_num), {}).get('quantity', 0)
                                                  for (wh_id, prod_id, b_num), _ in current_inventory.items()
                                                  if wh_id == warehouse_id and prod_id == product_id)

                if remaining_to_sell > total_available_for_product:
                    stockout_quantity = remaining_to_sell - total_available_for_product
                    stockout_records.append({
                        'stockout_date': current_date,
                        'warehouse_id': warehouse_id,
                        'product_id': product_id,
                        'stockout_quantity': stockout_quantity
                    })
                    remaining_to_sell = total_available_for_product # Only sell what's available

                for expiry_date_batch, batch_num, batch_quantity in available_batches:
                    key = (warehouse_id, product_id, batch_num)
                    if remaining_to_sell <= 0:
                        break

                    if current_inventory[key]['quantity'] >= remaining_to_sell:
                        current_inventory[key]['quantity'] -= remaining_to_sell
                        remaining_to_sell = 0
                    else:
                        remaining_to_sell -= current_inventory[key]['quantity']
                        current_inventory[key]['quantity'] = 0

        # Record current inventory state for this date
        for (wh_id, prod_id, batch_num), details in current_inventory.items():
            if details['quantity'] > 0:
                inventory_records.append({
                    'snapshot_date': current_date,
                    'warehouse_id': wh_id,
                    'product_id': prod_id,
                    'batch_number': batch_num,
                    'quantity_on_hand': details['quantity'],
                    'expiry_date': details['expiry_date']
                })

    inventory_df = pd.DataFrame(inventory_records)
    inventory_df['snapshot_date'] = pd.to_datetime(inventory_df['snapshot_date'])
    inventory_df['expiry_date'] = pd.to_datetime(inventory_df['expiry_date'])
    inventory_df = inventory_df.sort_values(by=['snapshot_date', 'warehouse_id', 'product_id', 'expiry_date']).reset_index(drop=True)
    inventory_df = inventory_df.drop_duplicates(subset=['snapshot_date', 'warehouse_id', 'product_id', 'batch_number'], keep='last')

    waste_df = pd.DataFrame(waste_records)
    if not waste_df.empty:
        waste_df['waste_date'] = pd.to_datetime(waste_df['waste_date'])
        waste_df['reason'] = waste_df['reason'].astype('category')

    stockout_df = pd.DataFrame(stockout_records)
    if not stockout_df.empty:
        stockout_df['stockout_date'] = pd.to_datetime(stockout_df['stockout_date'])

    return inventory_df, waste_df, stockout_df

In [63]:
if __name__ == '__main__':

    NUM_PRODUCTS = 5000
    NUM_WAREHOUSES = 10
    NUM_SUPPLIERS = 50
    NUM_CUSTOMERS = 1000
    NUM_PURCHASE_ORDERS = 10000
    NUM_SALES_RECORDS = 50000
    NUM_RETURNS = int(NUM_SALES_RECORDS * 0.05) # 5% of sales are returned

    START_DATE = datetime.now() - timedelta(days=2*365)
    END_DATE = datetime.now()


    products_df = generate_products_df(NUM_PRODUCTS, fake)
    print("products_df")
    print(products_df.head())

    warehouses_df = generate_warehouses_df(NUM_WAREHOUSES)
    print("\nwarehouses_df")
    print(warehouses_df.head())

    suppliers_df = generate_suppliers_df(NUM_SUPPLIERS, fake)
    print("\nsuppliers_df")
    print(suppliers_df.head())

    purchase_orders_df = generate_purchase_orders_df(NUM_PURCHASE_ORDERS, products_df, suppliers_df, START_DATE, END_DATE)
    print("\npurchase_orders_df")
    print(purchase_orders_df.head())

    customers_df = generate_customers_df(NUM_CUSTOMERS, fake)
    print("\ncustomers_df")
    print(customers_df.head())

    sales_df = generate_sales_df(NUM_SALES_RECORDS, products_df, customers_df, warehouses_df, START_DATE, END_DATE)
    print("\nsales_df")
    print(sales_df.head())

    returns_df = generate_returns_df(sales_df, NUM_RETURNS)
    print("\nreturns_df")
    print(returns_df.head())

    inventory_df, waste_df, stockout_df = simulate_inventory_and_waste(purchase_orders_df, sales_df, returns_df, products_df, warehouses_df, START_DATE, END_DATE)
    print("\ninventory_df")
    print(inventory_df.head())

    if not waste_df.empty:
        print("\nwaste_df (Spoilage)")
        print(waste_df.head())
    else:
        print("\nNo Waste (Spoilage) records generated.")

    if not stockout_df.empty:
        print("\nstockout_df")
        print(stockout_df.head())
    else:
        print("\nNo Stockout records generated.")


products_df
   product_id                product_name   product_category  unit_price  \
0  PROD_00001      Capsule player account   Gastrointestinal       20.82   
1  PROD_00002  Ointment energy technology     Pain Relievers       14.39   
2  PROD_00003          MP-857 unit player    Herbal Remedies       16.28   
3  PROD_00004      Capsule reduce medical           Hormonal       37.71   
4  PROD_00005         Drop senior mention  Ophthalmic & Otic       23.17   

   avg_shelf_life_days storage_conditions  
0                  730       Cold Storage  
1                  730   Room Temperature  
2                 1095       Cold Storage  
3                  730   Room Temperature  
4                 1460   Room Temperature  

warehouses_df
  warehouse_id             warehouse_name warehouse_city  latitude  longitude  \
0        WH_01   Central Warehouse Tehran         Tehran   35.6892    51.3890   
1        WH_02  Central Warehouse Mashhad        Mashhad   36.2605    59.6168   
2        

In [ ]:
# CSV (for easy viewing)
    products_df.to_csv('products.csv', index=False)
    warehouses_df.to_csv('warehouses.csv', index=False)
    suppliers_df.to_csv('suppliers.csv', index=False)
    customers_df.to_csv('customers.csv', index=False)
    purchase_orders_df.to_csv('purchase_orders.csv', index=False)
    sales_df.to_csv('sales.csv', index=False)
    returns_df.to_csv('returns.csv', index=False)
    inventory_df.to_csv('inventory.csv', index=False)
    waste_df.to_csv('waste.csv', index=False)
    stockout_df.to_csv('stockouts.csv', index=False)


    products_df.to_parquet('products.parquet', index=False)
    warehouses_df.to_parquet('warehouses.parquet', index=False)
    suppliers_df.to_parquet('suppliers.parquet', index=False)
    customers_df.to_parquet('customers.parquet', index=False)
    purchase_orders_df.to_parquet('purchase_orders.parquet', index=False)
    sales_df.to_parquet('sales.parquet', index=False)
    returns_df.to_parquet('returns.parquet', index=False)
    inventory_df.to_parquet('inventory.parquet', index=False)
    waste_df.to_parquet('waste.parquet', index=False)
    stockout_df.to_parquet('stockouts.parquet', index=False)